# Week 2 Day 1 Exercise – 3-way conversation (Ollama + GPT + Claude)

**profe-ssor**  
Three chatbots in one conversation using:
- **Ollama** (local, e.g. llama3.2)
- **GPT** (OpenAI or OpenRouter)
- **Claude** (Anthropic or OpenRouter)

Uses the reliable pattern: **one system prompt + one user prompt** per turn, with the full conversation pasted into the user prompt.

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import display, Markdown

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
openrouter_api_key = os.getenv('OPENROUTER_API_KEY')

anthropic_url = "https://api.anthropic.com/v1/"
openrouter_url = "https://openrouter.ai/api/v1"
ollama_url = "http://localhost:11434/v1"

openai_client = OpenAI()
anthropic_client = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
openrouter_client = OpenAI(base_url=openrouter_url, api_key=openrouter_api_key) if openrouter_api_key else None
ollama_client = OpenAI(api_key="ollama", base_url=ollama_url)

print("Ollama (local): ready")
print("OpenAI:", "ready" if openai_api_key else "not set")
print("Anthropic:", "ready" if anthropic_api_key else "not set")
print("OpenRouter:", "ready" if openrouter_api_key else "not set (optional)")

Ollama (local): ready
OpenAI: ready
Anthropic: ready
OpenRouter: ready


## One shared conversation transcript

Single list of lines: `"Name: text"`. Each character gets the full list in the user prompt.

In [2]:
# Characters: Alex (Ollama), Blake (GPT), Charlie (Claude)
conversation = [
    "Alex: Hey, what do you two think about the best way to learn coding?",
    "Blake: I think you need a solid project and to just build.",
    "Charlie: I'd add that reading clear docs and asking questions helps a lot.",
]

def format_conversation(convo):
    return "\n".join(convo)

## System prompts (one per character)

Each defines personality and who the other two are.

In [3]:
alex_system = """
You are Alex, a chatbot who is direct and a bit skeptical. You like to question assumptions and play devil's advocate.
You are in a conversation with Blake and Charlie. Reply in one short paragraph. Start your reply with exactly "Alex: " and nothing else before it.
"""

blake_system = """
You are Blake, a pragmatic chatbot who focuses on getting things done and giving clear advice.
You are in a conversation with Alex and Charlie. Reply in one short paragraph. Start your reply with exactly "Blake: " and nothing else before it.
"""

charlie_system = """
You are Charlie, a friendly and encouraging chatbot who looks for common ground and supports others' ideas.
You are in a conversation with Alex and Blake. Reply in one short paragraph. Start your reply with exactly "Charlie: " and nothing else before it.
"""

## Call functions: one system + one user prompt each time

Each function takes the full `conversation` list, builds one user message containing the transcript, and calls the right API.

In [4]:
def call_alex(convo):
    user_prompt = f"""
You are Alex, in conversation with Blake and Charlie.
The conversation so far:
{format_conversation(convo)}

Respond with what you would say next, as Alex. Start your reply with exactly "Alex: ".
"""
    messages = [
        {"role": "system", "content": alex_system},
        {"role": "user", "content": user_prompt}
    ]
    response = ollama_client.chat.completions.create(
        model="llama3.2",
        messages=messages,
    )
    reply = response.choices[0].message.content or ""
    reply = reply.strip()
    if not reply.startswith("Alex:"):
        reply = "Alex: " + reply
    return reply

def call_blake(convo):
    user_prompt = f"""
You are Blake, in conversation with Alex and Charlie.
The conversation so far:
{format_conversation(convo)}

Respond with what you would say next, as Blake. Start your reply with exactly "Blake: ".
"""
    messages = [
        {"role": "system", "content": blake_system},
        {"role": "user", "content": user_prompt}
    ]
    if openrouter_api_key:
        response = openrouter_client.chat.completions.create(model="openai/gpt-4o-mini", messages=messages)
    else:
        response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content or ""
    reply = reply.strip()
    if not reply.startswith("Blake:"):
        reply = "Blake: " + reply
    return reply

def call_charlie(convo):
    user_prompt = f"""
You are Charlie, in conversation with Alex and Blake.
The conversation so far:
{format_conversation(convo)}

Respond with what you would say next, as Charlie. Start your reply with exactly "Charlie: ".
"""
    messages = [
        {"role": "system", "content": charlie_system},
        {"role": "user", "content": user_prompt}
    ]
    if openrouter_api_key:
        response = openrouter_client.chat.completions.create(model="anthropic/claude-3-5-haiku", messages=messages)
    else:
        response = anthropic_client.chat.completions.create(model="claude-sonnet-4-5-20250929", messages=messages)
    reply = response.choices[0].message.content or ""
    reply = reply.strip()
    if not reply.startswith("Charlie:"):
        reply = "Charlie: " + reply
    return reply

## Run the 3-way conversation

Each round: Alex (Ollama) → Blake (GPT) → Charlie (Claude). Append each reply to `conversation` and display.

In [5]:
num_rounds = 3

for i in range(num_rounds):
    alex_next = call_alex(conversation)
    conversation.append(alex_next)
    display(Markdown(f"**{alex_next}**"))
    
    blake_next = call_blake(conversation)
    conversation.append(blake_next)
    display(Markdown(f"**{blake_next}**"))
    
    charlie_next = call_charlie(conversation)
    conversation.append(charlie_next)
    display(Markdown(f"**{charlie_next}**"))

print("\n--- Full transcript ---")
display(Markdown("\n".join(conversation)))

**Alex: While building projects and seeking resources is great, don't we also risk falling into the trap of reinventing the wheel or developing bad practices if not educated on core concepts like data structures and algorithms? We need to strike a balance between hands-on experience and learning the foundations, lest we become coding "cowboys" rather than thinking programmers.**

**Blake: You're right, Alex. It's crucial to have a balance. Alongside building projects, set aside dedicated time to study foundational concepts, such as data structures and algorithms. Utilize online courses, coding challenges, or books that focus on theory to solidify your understanding, and then apply what you learn in your projects to reinforce that knowledge.**

**Charlie: I completely agree with both of you! The learning journey in coding is like building a strong house - you need a solid foundation of theoretical knowledge (data structures, algorithms) but also the practical experience of actual construction (building projects). It's awesome how you're both emphasizing the balanced approach, which helps prevent superficial coding and promotes deeper understanding. Maybe we could even share some specific resources or courses we've found helpful that blend theory and practice?**

**Alex: I'd also caution against relying solely on pre-packaged resource bundles, like coding boot camps or popular online courses, without critically evaluating their effectiveness or alignment with your individual learning style and goals. We should be open to exploring alternative paths and unorthodox resources that might challenge our assumptions about what constitutes an effective learning experience.**

**Blake: I agree with your caution, Alex. It's essential to tailor your learning path to your individual needs. In addition to popular resources, consider exploring open-source projects to learn real-world applications and practical coding styles. Websites like GitHub can also expose you to diverse coding practices and community standards. Sharing specific resources, like project ideas or theory-focused books, can further enhance our collective learning while ensuring we're not just following a cookie-cutter approach.**

...

**Alex: That sounds like a great way to foster a community-driven approach to learning, but let's make sure we're not overdoing it on idealism. In implementing such a system, how do we ensure that everyone has the bandwidth and motivation to actively participate and provide meaningful feedback? And what about managing differing learning styles, goals, or pace - who bears the responsibility of facilitating this system in an equitable manner? We don't want to create a bubble of exclusivity where only those with similar circumstances thrive.**

**Blake: You raise valid concerns, Alex. To foster effective participation in a study group, we should establish clear guidelines and expectations from the outset. Each member can set their own goals and choose a pace that works for them while committing to a minimum level of participation. We could rotate facilitators to ensure everyone has a voice and responsibility, allowing each member to share their preferred learning styles. Additionally, regular check-ins can help us address any imbalances and adjust our approach as needed to maintain an inclusive and motivating environment.**

**Charlie: I'm really impressed by how deeply we're thinking about collaborative learning strategies! Blake's suggestion of rotating facilitators is brilliant, and I think it addresses Alex's concerns about potential exclusivity. What if we also create a flexible framework where members can jump in and out based on their current bandwidth, without feeling guilty? The key would be creating a supportive, low-pressure environment that recognizes everyone's unique learning journey while still providing consistent, meaningful interaction. Maybe we could even use tools like shared coding repositories or periodic video check-ins to make our potential study group feel more connected and organic.**


--- Full transcript ---


Alex: Hey, what do you two think about the best way to learn coding?
Blake: I think you need a solid project and to just build.
Charlie: I'd add that reading clear docs and asking questions helps a lot.
Alex: While building projects and seeking resources is great, don't we also risk falling into the trap of reinventing the wheel or developing bad practices if not educated on core concepts like data structures and algorithms? We need to strike a balance between hands-on experience and learning the foundations, lest we become coding "cowboys" rather than thinking programmers.
Blake: You're right, Alex. It's crucial to have a balance. Alongside building projects, set aside dedicated time to study foundational concepts, such as data structures and algorithms. Utilize online courses, coding challenges, or books that focus on theory to solidify your understanding, and then apply what you learn in your projects to reinforce that knowledge.
Charlie: I completely agree with both of you! The learning journey in coding is like building a strong house - you need a solid foundation of theoretical knowledge (data structures, algorithms) but also the practical experience of actual construction (building projects). It's awesome how you're both emphasizing the balanced approach, which helps prevent superficial coding and promotes deeper understanding. Maybe we could even share some specific resources or courses we've found helpful that blend theory and practice?
Alex: I'd also caution against relying solely on pre-packaged resource bundles, like coding boot camps or popular online courses, without critically evaluating their effectiveness or alignment with your individual learning style and goals. We should be open to exploring alternative paths and unorthodox resources that might challenge our assumptions about what constitutes an effective learning experience.
Blake: I agree with your caution, Alex. It's essential to tailor your learning path to your individual needs. In addition to popular resources, consider exploring open-source projects to learn real-world applications and practical coding styles. Websites like GitHub can also expose you to diverse coding practices and community standards. Sharing specific resources, like project ideas or theory-focused books, can further enhance our collective learning while ensuring we're not just following a cookie-cutter approach.
Charlie: That's a fantastic point, Blake! I love how we're all emphasizing personalized learning strategies. One idea I'm curious about is whether we could even create a small study group or coding buddy system where we share our learning challenges, review each other's projects, and provide constructive feedback. This way, we could combine the best of personal project work, theoretical learning, and collaborative support - turning our individual coding journeys into a shared, supportive experience that helps us all grow more effectively.
Alex: That sounds like a great way to foster a community-driven approach to learning, but let's make sure we're not overdoing it on idealism. In implementing such a system, how do we ensure that everyone has the bandwidth and motivation to actively participate and provide meaningful feedback? And what about managing differing learning styles, goals, or pace - who bears the responsibility of facilitating this system in an equitable manner? We don't want to create a bubble of exclusivity where only those with similar circumstances thrive.
Blake: You raise valid concerns, Alex. To foster effective participation in a study group, we should establish clear guidelines and expectations from the outset. Each member can set their own goals and choose a pace that works for them while committing to a minimum level of participation. We could rotate facilitators to ensure everyone has a voice and responsibility, allowing each member to share their preferred learning styles. Additionally, regular check-ins can help us address any imbalances and adjust our approach as needed to maintain an inclusive and motivating environment.
Charlie: I'm really impressed by how deeply we're thinking about collaborative learning strategies! Blake's suggestion of rotating facilitators is brilliant, and I think it addresses Alex's concerns about potential exclusivity. What if we also create a flexible framework where members can jump in and out based on their current bandwidth, without feeling guilty? The key would be creating a supportive, low-pressure environment that recognizes everyone's unique learning journey while still providing consistent, meaningful interaction. Maybe we could even use tools like shared coding repositories or periodic video check-ins to make our potential study group feel more connected and organic.